# Métricas do câmbio do dólar (2025)

## About

Este projeto tem como objetivo gerar estatísticas básicas para analisar a flutuação de valor do dólar em relação ao real durante o ano de 2025. Para isso, foram utilizadas técnicas de webscraping para obter a série histórica do dólar para o ano. Uma vez obtidos, os valores foram adicionados a uma planilha excel salva em nuvem, que por sua vez está vinculada a um dashboard Google Looker, onde foram geradas as estatísticas. 

O projeto está estruturado de forma a sempre os últimos valores divulgados, portanto, para atualizar o dashboard, basta rodar o código abaixo:


In [5]:
import requests
import pandas as pd
from datetime import datetime
import lxml
import openpyxl

### Definindo as variáveis:

In [14]:
URL = "https://idealsoftwares.com.br/indices/dolar2025.html"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; GitHub Copilot)"}
wd = "G:\\Meu Drive\\Eric\\Educação\\MBA_USP\\dsc_data"

### Definindo as funções:



In [16]:
def busca_tabela(url):
    resp = requests.get(url, headers=HEADERS, timeout=15)
    resp.raise_for_status()
    resp.encoding = "utf-8"
    tabelas = pd.read_html(resp.text)
    return tabelas

def pick_tabela(tabelas):
    if not tabelas:
        raise ValueError("Não foi possível encontrar tabelas na página.")
    # Trará a primeira tabela do array. Como já foi analisada a página destino, é sabido que há apenas uma tabela, portanto não há a necessidade de ajustes.
    return tabelas[0]

def clean_table(df: pd.DataFrame) -> pd.DataFrame:
    # basic clean: strip string columns and drop fully empty columns/rows
    df = df.dropna(axis=1, how="all").dropna(axis=0, how="all")
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.strip()
    return df

In [38]:
if __name__ == "__main__":
    tabelas = busca_tabela(URL)
    df = pick_tabela(tabelas)
    df = clean_table(df)
    out_file = df.to_excel(f"{wd}\\USD2025_stats.xlsx", index=False)
    print("Planilha salva em:", wd)


Planilha salva em: G:\Meu Drive\Eric\Educação\MBA_USP\dsc_data


C:\Users\Mr_Li\AppData\Local\Temp\ipykernel_41888\2248515830.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas = pd.read_html(resp.text)


In [ ]:
# Para conferir se o arquivo foi salvo corretamente, podemos imprimir as primeiras linhas da tabela com:

print(df.head())

# Algumas estatísticas básicas sobre a coluna "Dólar Venda":
print("Média: ", df["Dólar Venda"].mean())
print("Mediana: ", df["Dólar Venda"].median())
print("Mínimo: ", df["Dólar Venda"].min())
print("Máximo: ", df["Dólar Venda"].max())



         Data  Dólar Compra  Dólar Venda
0  01/10/2025        5.3202       5.3208
1  30/09/2025        5.3180       5.3186
2  29/09/2025        5.3223       5.3229
3  26/09/2025        5.3439       5.3445
4  25/09/2025        5.3419       5.3425
Média:  5.648441798941799
Mediana:  5.6539
Mínimo:  5.301
Máximo:  6.2086
